In [1]:
SCO_csv = "./Scopus/scopus.csv"
SD_csv = "./ScienceDirect/ScienceDirect.csv"
WOS_csv  = "./WOS/wos.csv"

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os
from glob import glob

In [3]:
shared_columns = ["Article Title", "Authors", "Volume", "Source Title", "Publication Year", "ISSN", "DOI"]

In [4]:
SCO_df = pd.read_csv(SCO_csv)
SD_df = pd.read_csv(SD_csv)
WOS_df = pd.read_csv(WOS_csv)

In [5]:
# Add a source column to each DataFrame to indicate the origin
SCO_df["Source"] = "SCO"
SD_df["Source"] = "SD"
WOS_df["Source"] = "WOS"

In [6]:
# Select only the shared columns plus the Source column from each DataFrame
SCO_selected = SCO_df[shared_columns + ["Source"]]
SD_selected = SD_df[shared_columns + ["Source"]]
WOS_selected = WOS_df[shared_columns + ["Source"]]

In [7]:
# Concatenate the DataFrames
All_df = pd.concat([SCO_selected, SD_selected, WOS_selected], ignore_index=True)

In [9]:
All_df.count()

Article Title       1891
Authors             1891
Volume              1734
Source Title        1889
Publication Year    1891
ISSN                1739
DOI                 1837
Source              1891
dtype: int64